In [1]:
import pandas as pd

In [25]:
def addCytobandTo_STR(str_df, cb_df):
    """Adds the cytoband the STR belongs to the STR dataframe as a new column
    """
    cytobands = []
    for row in str_df.index:
        cytoband_found = False
        f_index = str_df["FirstIndex"][row]
        l_index = str_df["LastIndex"][row]
        chrom = str_df["FastaHeader"][row]
        cb_chrdf = cb_df.loc[cb_df["Chromossome"] == chrom]
        for line in cb_chrdf.index:
            if int(cb_chrdf["First_index"][line]) < int(f_index) and int(cb_chrdf["Last_index"][line])-1 > int(l_index) and not cytoband_found:
                cytobands.append(cb_chrdf["Citoband"][line])
                cytoband_found = True
        if not cytoband_found:
            for cb_index in cb_chrdf.index:
                if int(cb_chrdf["First_index"][cb_index]) < int(f_index) and int(cb_chrdf["Last_index"][cb_index+1])-1 > int(l_index) and not cytoband_found:
                    cytobands.append(cb_chrdf["Citoband"][cb_index] + " & " + cb_chrdf["Citoband"][cb_index+1])
                    cytoband_found = True
            if not cytoband_found:
                    cytobands.append("cytoband not found")
    
    str_df.insert(len(str_df.columns), "Cytoband", cytobands, True)

### Completar a versão melhorada da função 

In [ ]:
#Verificar
def addCytobandTo_STR2(str_df, cb_df):
    """Adds the cytoband the STR belongs to the STR dataframe as a new column
    """
    cytobands = []
    for row in str_df.index:
        cytoband_found = False
        f_index = str_df["FirstIndex"][row]
        l_index = str_df["LastIndex"][row]
        chrom = str_df["FastaHeader"][row]
        cb_chrdf = cb_df.loc[cb_df["Chromossome"] == chrom]
        ### STR só numa cytoband
        str_cytoband = cb_chrdf.loc[(cb_chrdf["First_index"] < int(f_index)) & (cb_chrdf["Last_index"] > int(l_index))]
        ### STR em mais que uma cytoband
        #Primeiro Cytoband
        str_cytobands_start = cb_chrdf.loc[(cb_chrdf["First_index"] < int(f_index)) & (cb_chrdf["Last_index"] < int(l_index))]
        #Ultimo Cytoband
        str_cytobands_finish = cb_chrdf.loc[(cb_chrdf["First_index"] > int(f_index)) & (cb_chrdf["Last_index"] > int(l_index))]
        #Cytoband intermedio
        str_cytobands_intermidiate = cb_chrdf.loc[(cb_chrdf["First_index"] > int(f_index)) & (cb_chrdf["Last_index"] < int(l_index))]
        if str_cytoband.index.size > 0:
            cytobands.append(str_cytoband["Citoband"][0])
        elif str_cytobands_start.index.size and str_cytobands_finish.index.size:
            cbs = str_cytobands_start["Citoband"][0] + "and" + str_cytobands_finish["Citoband"][0]
            cytobands.append(cbs)
        else:
            cytobands.append("cytoband not found")
    
    str_df.insert(len(str_df.columns), "Cytoband", cytobands, True)

In [3]:
def writeNewtxt(file_path, df):
    """Writes a pandas dataframe into txt, columns separated by tabs ("\t")
    """
    f = open(file_path, 'w')
    str_cols = ""
    for col in df.columns[:-1]:
        str_cols += col + "\t"
    str_cols += df.columns.values[-1]+ "\n"
    f.write(str(str_cols))
    for row in df.values:
        str_row = ""
        for col in row[:-1]:
            str_row += str(col) + "\t"
        str_row += str(row[-1])+ "\n"
        f.write(str_row)
    f.close

In [14]:
#CytobandFile = "/home/androx/Documents/trabalho/citobands/cytobandFiltered.txt"
CytobandFile = "/home/androx/Documents/trabalho/citobands/cytobandFiltered_processed_complete.txt"
#STRPath_read = "/home/androx/Documents/trabalho/datasets/STR/HomosapiensHG38_nov2014/HomosapiensHG38_FullGenome.txt"
#STRPath_write = "/home/androx/Documents/trabalho/datasets/STR/HomosapiensHG38_nov2014/HomosapiensHG38_FullGenome_processed.txt"
STRPath_read = "/home/androx/Documents/trabalho/datasets/STR/HomosapiensHG38_jan2016_latest/HomosapiensHG38_FullGenome.txt"
STRPath_write = "/home/androx/Documents/trabalho/datasets/STR/HomosapiensHG38_jan2016_latest/HomosapiensHG38_FullGenome_processed.txt"

In [19]:
citobands_df = pd.read_csv(CytobandFile, sep="\t")#, header=None)
str_df = str_dataframe = pd.read_csv(STRPath_read, sep="\t")

In [20]:
citobands_df

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,STR_Density
0,chr1,0,2299999,p36.33,gneg,2299999,0.088
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.056
...,...,...,...,...,...,...,...
858,chrY,12400000,17099999,q11.221,gpos50,4699999,0.037
859,chrY,17100000,19599999,q11.222,gneg,2499999,0.062
860,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082
861,chrY,23800000,26599999,q11.23,gneg,2799999,0.026


In [7]:
str_df

,Repeatid,FirstIndex,LastIndex,PatternSize,CopyNumber,MatchPerc,MisMatchPerc,IndelPerc,ArrayLength,FastaHeader,FlankingLeft50,Pattern,ArraySequence,FlankingRight50,SequenceId
0,185489081,10001,10983,6,162.444443,90,8,2,983,chr3,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,CTAACC,CTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAA...,CTCTgcaggcgcagagaggcgcgcggcgccggcgcaggcgcagaga...,29622332
1,185561875,10001,10197,6,32.857143,97,1,2,197,chr4,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,ACCCTA,ACCCTAACCCTAACCCTAACCCTAACCCTAACCCTACCCTAACCCT...,Cattattctcggctgcaaagaggaaggatctttaccgtggatgtgg...,29622333
2,185634287,10001,11835,6,305.000000,99,1,0,1835,chr5,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,TAACCC,TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAAC...,ctgacccctgaccccgaccccgaccccgaccccgaccctgacccTA...,29622334
3,185767514,10001,10241,6,42.166668,94,2,4,241,chr7,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,CTAACC,CTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAA...,AACATTCAAAAGCTGAGCAGGGCTTTAAAGCTATCTTATTAATAAT...,29622336
4,185893557,10001,10472,6,77.500000,88,7,5,472,chr9,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,TAACCC,TAACCCTAACCCTAACCCTAACCCAACCCCACCCCAACCCCAACCC...,gcacatgaggaatgtgggtgttatattttgggtatcatgtgtgcat...,29622338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199623,184914593,248944941,248945034,29,3.275862,98,1,1,94,chr1,ggaggcgccggcgtgcgtctatgccctgcgccgcgtctccccaaca...,CGCGCCGCGCCTCTCTGCGCCTGCGCCGG,CGCGCGCGCCTCTCTGCGCCTGCGCCGGCGCGCCGCGCCTCTCTGC...,ctttgcgagggcggagttgcgttctctttagcacacacccggagag...,29622330
1199624,184914592,248944954,248945029,6,13.000000,83,10,7,76,chr1,tgcgtctatgccctgcgccgcgtctccccaacagcgccgcgcgcgc...,CTGCGC,CTGCGCCTGCGCCGGCGCGCCGCGCCTCTCTGCGCCTGCGCCGGCG...,gccgcctttgcgagggcggagttgcgttctctttagcacacacccg...,29622330
1199625,184914594,248945037,248945157,47,2.617021,86,13,1,121,chr1,gcctgcgccggcgcgccgcgcctctctgcgcctgcgccggcgcgcc...,TTGCGAGGGCGGAGCTGCGTTCTCCTCAGCACACACTCCGGAGAGCA,TTGCGAGGGCGGAGTTGCGTTCTCTTTAGCACACACCCGGAGAGCA...,gtcagacctgggcgggcgggctgagggcactgcgagggcggagctg...,29622330
1199626,184914595,248945549,248946422,6,141.800003,92,4,4,874,chr1,ttcctaatgcacacatgacacccaaaatataacagacatattactc...,AGGGTT,AGGGTGAGGGTGAGGGTTCGGGTTAGGGTTCGGGTTAGGGTTCGGG...,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,29622330


In [10]:
str_df = str_df.drop(["MatchPerc", "MisMatchPerc", "IndelPerc"], axis=1)

In [11]:
str_df

,Repeatid,FirstIndex,LastIndex,PatternSize,CopyNumber,ArrayLength,FastaHeader,FlankingLeft50,Pattern,ArraySequence,FlankingRight50,SequenceId
0,185489081,10001,10983,6,162.444443,983,chr3,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,CTAACC,CTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAA...,CTCTgcaggcgcagagaggcgcgcggcgccggcgcaggcgcagaga...,29622332
1,185561875,10001,10197,6,32.857143,197,chr4,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,ACCCTA,ACCCTAACCCTAACCCTAACCCTAACCCTAACCCTACCCTAACCCT...,Cattattctcggctgcaaagaggaaggatctttaccgtggatgtgg...,29622333
2,185634287,10001,11835,6,305.000000,1835,chr5,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,TAACCC,TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAAC...,ctgacccctgaccccgaccccgaccccgaccccgaccctgacccTA...,29622334
3,185767514,10001,10241,6,42.166668,241,chr7,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,CTAACC,CTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAA...,AACATTCAAAAGCTGAGCAGGGCTTTAAAGCTATCTTATTAATAAT...,29622336
4,185893557,10001,10472,6,77.500000,472,chr9,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,TAACCC,TAACCCTAACCCTAACCCTAACCCAACCCCACCCCAACCCCAACCC...,gcacatgaggaatgtgggtgttatattttgggtatcatgtgtgcat...,29622338
...,...,...,...,...,...,...,...,...,...,...,...,...
1199623,184914593,248944941,248945034,29,3.275862,94,chr1,ggaggcgccggcgtgcgtctatgccctgcgccgcgtctccccaaca...,CGCGCCGCGCCTCTCTGCGCCTGCGCCGG,CGCGCGCGCCTCTCTGCGCCTGCGCCGGCGCGCCGCGCCTCTCTGC...,ctttgcgagggcggagttgcgttctctttagcacacacccggagag...,29622330
1199624,184914592,248944954,248945029,6,13.000000,76,chr1,tgcgtctatgccctgcgccgcgtctccccaacagcgccgcgcgcgc...,CTGCGC,CTGCGCCTGCGCCGGCGCGCCGCGCCTCTCTGCGCCTGCGCCGGCG...,gccgcctttgcgagggcggagttgcgttctctttagcacacacccg...,29622330
1199625,184914594,248945037,248945157,47,2.617021,121,chr1,gcctgcgccggcgcgccgcgcctctctgcgcctgcgccggcgcgcc...,TTGCGAGGGCGGAGCTGCGTTCTCCTCAGCACACACTCCGGAGAGCA,TTGCGAGGGCGGAGTTGCGTTCTCTTTAGCACACACCCGGAGAGCA...,gtcagacctgggcgggcgggctgagggcactgcgagggcggagctg...,29622330
1199626,184914595,248945549,248946422,6,141.800003,874,chr1,ttcctaatgcacacatgacacccaaaatataacagacatattactc...,AGGGTT,AGGGTGAGGGTGAGGGTTCGGGTTAGGGTTCGGGTTAGGGTTCGGG...,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,29622330


In [26]:
addCytobandTo_STR(str_df, citobands_df)

In [27]:
str_df

,Repeatid,FirstIndex,LastIndex,PatternSize,CopyNumber,MatchPerc,MisMatchPerc,IndelPerc,ArrayLength,FastaHeader,FlankingLeft50,Pattern,ArraySequence,FlankingRight50,SequenceId,Cytoband
0,185489081,10001,10983,6,162.444443,90,8,2,983,chr3,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,CTAACC,CTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAA...,CTCTgcaggcgcagagaggcgcgcggcgccggcgcaggcgcagaga...,29622332,p26.3
1,185561875,10001,10197,6,32.857143,97,1,2,197,chr4,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,ACCCTA,ACCCTAACCCTAACCCTAACCCTAACCCTAACCCTACCCTAACCCT...,Cattattctcggctgcaaagaggaaggatctttaccgtggatgtgg...,29622333,p16.3
2,185634287,10001,11835,6,305.000000,99,1,0,1835,chr5,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,TAACCC,TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAAC...,ctgacccctgaccccgaccccgaccccgaccccgaccctgacccTA...,29622334,p15.33
3,185767514,10001,10241,6,42.166668,94,2,4,241,chr7,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,CTAACC,CTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAA...,AACATTCAAAAGCTGAGCAGGGCTTTAAAGCTATCTTATTAATAAT...,29622336,p22.3
4,185893557,10001,10472,6,77.500000,88,7,5,472,chr9,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,TAACCC,TAACCCTAACCCTAACCCTAACCCAACCCCACCCCAACCCCAACCC...,gcacatgaggaatgtgggtgttatattttgggtatcatgtgtgcat...,29622338,p24.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199623,184914593,248944941,248945034,29,3.275862,98,1,1,94,chr1,ggaggcgccggcgtgcgtctatgccctgcgccgcgtctccccaaca...,CGCGCCGCGCCTCTCTGCGCCTGCGCCGG,CGCGCGCGCCTCTCTGCGCCTGCGCCGGCGCGCCGCGCCTCTCTGC...,ctttgcgagggcggagttgcgttctctttagcacacacccggagag...,29622330,q44
1199624,184914592,248944954,248945029,6,13.000000,83,10,7,76,chr1,tgcgtctatgccctgcgccgcgtctccccaacagcgccgcgcgcgc...,CTGCGC,CTGCGCCTGCGCCGGCGCGCCGCGCCTCTCTGCGCCTGCGCCGGCG...,gccgcctttgcgagggcggagttgcgttctctttagcacacacccg...,29622330,q44
1199625,184914594,248945037,248945157,47,2.617021,86,13,1,121,chr1,gcctgcgccggcgcgccgcgcctctctgcgcctgcgccggcgcgcc...,TTGCGAGGGCGGAGCTGCGTTCTCCTCAGCACACACTCCGGAGAGCA,TTGCGAGGGCGGAGTTGCGTTCTCTTTAGCACACACCCGGAGAGCA...,gtcagacctgggcgggcgggctgagggcactgcgagggcggagctg...,29622330,q44
1199626,184914595,248945549,248946422,6,141.800003,92,4,4,874,chr1,ttcctaatgcacacatgacacccaaaatataacagacatattactc...,AGGGTT,AGGGTGAGGGTGAGGGTTCGGGTTAGGGTTCGGGTTAGGGTTCGGG...,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,29622330,q44


In [28]:
writeNewtxt(STRPath_write, str_df)

In [29]:
str_df_cb_notfound = str_df.loc[str_df['Cytoband'] == "cytoband not found"]
str_df_cb_notfound

,Repeatid,FirstIndex,LastIndex,PatternSize,CopyNumber,MatchPerc,MisMatchPerc,IndelPerc,ArrayLength,FastaHeader,FlankingLeft50,Pattern,ArraySequence,FlankingRight50,SequenceId,Cytoband
111940,185453577,10864561,12915808,1364,1504.000000,85,14,1,2051248,chr21,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,AGCATTCTCAGAAACTTCTTTGTGATGTGTGCACTCAACTAACAGA...,AGCATTCTGAGAAATTACTTTGTGATGTGTGCATTCATCACAAAGA...,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,29622350,cytoband not found
111980,185453576,10867958,12915808,171,12013.000000,71,27,2,2047851,chr21,gaatgggatatttctgagaagttggaggcctatatcgaaaaagaaa...,ATTCTCAGAAACTTCTTTGTGATGTGTGCATTCAACTCACAGAGTT...,ATTCTGAGAAACTTCTTTGAGATGTATCCTTTCATCTCACAGAATT...,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,29622350,cytoband not found
130406,185470427,13021432,15054318,171,11924.900391,72,25,3,2032887,chr22,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNAACAT...,AGAAACTTCTTTGTGATGTGTGCATTCAACTCACAGAGTTGAACAT...,AGAAACTGCTTTGTGATGCGTGCATTCATCACCAGTAGTTGAGTTT...,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,29622351,cytoband not found
130408,185470429,13021432,15054318,1363,1490.400024,85,14,1,2032887,chr22,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNAACAT...,AGAAACTTCTTTGTGATGTGTGCACTCAACTAACAGAGTTGAACCT...,AGAAACTGCTTTGTGATGCGTGCATTCATCACCAGTAGTTGAGTTT...,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,29622351,cytoband not found
155733,185076729,16000001,18051248,1364,1504.000000,85,14,1,2051248,chr13,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,AGCATTCTCAGAAACTTCTTTGTGATGTGTGCACTCAACTAACAGA...,AGCATTCTGAGAAATTACTTTGTGATGTGTGCATTCATCACAAAGA...,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,29622342,cytoband not found
155764,185076728,16003398,18051248,171,12013.000000,71,27,2,2047851,chr13,gaatgggatatttctgagaagttggaggcctatatcgaaaaagaaa...,ATTCTCAGAAACTTCTTTGTGATGTGTGCATTCAACTCACAGAGTT...,ATTCTGAGAAACTTCTTTGAGATGTATCCTTTCATCTCACAGAATT...,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,29622342,cytoband not found
165381,185149802,17083674,19725208,171,15472.700195,73,25,2,2641535,chr15,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,AGCATTCTGAGAAACTTCTTTGTGATGTGTGCATTCAACTCACAGA...,AGCATTCCGTAAAACTTCTTTGTGATGTGTGCATTCGTCTCACAGA...,aaatttcgttggaagcgggaaaaccttcatataaaatctagacagC...,29622344,cytoband not found
165383,185149803,17083685,19725208,513,5162.600098,77,22,1,2641524,chr15,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNAGcattc...,AAACTTCTTTGTGATGTGTGCATTCAACTCACAGAGTTGAACCTTT...,AAACTTCTTTGTGATGTGTGCATTCGTCTCACAGAGTTGAACCTAT...,aaatttcgttggaagcgggaaaaccttcatataaaatctagacagC...,29622344,cytoband not found


In [38]:
str_df_cb_notfound = str_df_cb_notfound.drop(["MatchPerc", "MisMatchPerc", "IndelPerc"], axis=1)

In [35]:
str_df_cb_notfound["ArraySequence"][111940][:1000]

'AGCATTCTGAGAAATTACTTTGTGATGTGTGCATTCATCACAAAGAGTTGAACCTTTCTTTTGGTTGAGCAGTTTTGAAGCACTCTTTTTGTAGAATCTGTAAGTGGATATTTGGAGTTCTTTGAGGCTTATGGTGGTAAAGGAAATATCTTCACACAAAAACTACACAGAACCATTCTGAAATACCTCTTTGTGATGCTTGCATTCATCTCACATAGTTGAACCATTCTTTTTATTGAGCAGTTTTGAAACAATCTCCTTGTAGAATGTGCAAGTGGATATTTGGAATGCTTTGATGAGTATGGTGGAAAATGAAAAATCTTCACATAAAAACTAGACAGAATTACTCTGAGAAACTTCTTTGTGATGTGCACATTCATCTCACAAATTTGAAAATTTCTTTTGATTGAGCAGTTTTGAAACGCTCTTTTTCTAGAATCTGCCAGTGGTTATTTGGAGTGCTTTGAGTCCTATGGTGGAGAAGGAAATATCCTCACATAAAAACTAGAGAGAAGCATTCTGAGAAACTTCTTTCTGATGTGTGCATACATCTCACGGAGTTGAAACTTTCTATTGATTTAGCATTTTTTATACACTTTTTGTAGGATCTGCAGTTGCTATTTGGAGCCCTTTGGGGCCAATGGTGGAAAAGTACTATCTTCTCATAAAAACTAGACAGAAGCATTCTGAGAAACTTCTTGGTGATGTGTGCATTCATCTCACAGTAGTTGAACCTTTCTTTTGATTGAGCAGTTTTGAAACGCTCTTTTCGTTGAATCTGCAAGTGCATATTTAGAGTGCTTTGAGGCACGTGGTGGAAAAGGAAATATCTTCACATAAACACTAGACAGAAGCATTCTGAGAAATGTCTTTGTGATGTGTCCATTCACTTCACAGAGTTGAAACTTTCTTTTCATTGAGCAGTTTTGAAACACTCTTTTTATAGAATATGCAAGTGGATATTTGGAGCGTTTTGGAGAGAATGGTGGAAATGGAAAT

In [39]:
str_cbNotFound_file = "/home/androx/Documents/trabalho/datasets/STR/HomosapiensHG38_jan2016_latest/HomosapiensHG38_XLSTRs.txt"
writeNewtxt(str_cbNotFound_file, str_df_cb_notfound)

## Fazer agora o calculo de densidades com 6 casas décimais ou mais se possivel paras os TR, STRs e LTRs